## Time-Series-Library

Time-Series-Library is used for this project.  
https://github.com/thuml/Time-Series-Library

When we find some files needed to be fixed/modified/added, then save the files in Time-Series-Library-patch directory, and override the original files with the patched files.

In [47]:
# Clone the repository of Time-Series-Library
!git clone https://github.com/thuml/Time-Series-Library.git

# Replace the original files with the patched files
!cp ./Time-Series-Library-patch/run.py Time-Series-Library/

fatal: destination path 'Time-Series-Library' already exists and is not an empty directory.


In [42]:
!pip install -r Time-Series-Library/requirements.txt

In [43]:
import sys

# Add Time-Series-Library directory to module lookup paths
sys.path.append('Time-Series-Library')

## Make dataset file of BTC-USD from yfinance

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

asset = "BTC-USD"

# Period of test data (TODO: Change the period)
start_date = "2020-01-01"
end_date = "2025-01-01"

# Daily data of BTC-USD is used 

# auto_adjust=True is applied by default. 
# So,'Close' price is used instead of 'Adj Close'
df = yf.download(asset, start=start_date, end=end_date, interval='1d')['Close']
df.columns = ['Close']

# Features (TODO: Change the features)
df["Ret"] = df["Close"].pct_change()
df["Ret_5"] = df['Close'].pct_change(periods=5)    
df["Ret_10"] = df['Close'].pct_change(periods=10)    
df["Ret_15"] = df['Close'].pct_change(periods=15)
df["Ret_20"] = df['Close'].pct_change(periods=20)
df["Ret_25"] = df['Close'].pct_change(periods=25)
df["Ret_30"] = df['Close'].pct_change(periods=30)
df["Ret_40"] = df['Close'].pct_change(periods=40)
df["Ret_50"] = df['Close'].pct_change(periods=50)    

# Target variable (TODO: Change the target variable)
df["OT"] = df["Ret_25"].shift(-25)  

del df["Close"]
df = df.dropna()

df.reset_index(inplace=True)
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
df.rename(columns={'Date': 'date'}, inplace=True)

# Save the dataset as csv file
dataset_path = 'dataset_BTC_r25.csv'
df.to_csv(path_or_buf=dataset_path, index=False)

df.head()


## Experiments with various models

NOTE:
 - short-term-forecast works only for M4 dataset! 
 - The experiment with the args below raises an error when testing with some models (TimeMixer, TimesNet, Mamba, ..) 

In [ ]:
# Set command line arguments common for all models
cmn_args  = "--task_name long_term_forecast --features MS --is_training 1 "
cmn_args += " --e_layers 2 --d_layers 1 --factor 3 --enc_in 1 --dec_in 1 --c_out 1 --d_model 512 --batch_size 16 "
cmn_args += " --seq_len 32 --label_len 32 --pred_len 1 --inverse "
cmn_args += " --itr 1 --train_epochs 1 --learning_rate 0.001 --des 'Exp' "
cmn_args += " --data custom --root_path ./ --data_path " + dataset_path

# Experiment with iTransformer
model = "iTransformer"
args = f"--model {model} --model_id {model}_BTC_lt " + cmn_args
!python ./Time-Series-Library/run.py $args

In [ ]:
# Experiment with TimeXer
model = "TimeXer"
args = f"--model {model} --model_id {model}_BTC_lt " + cmn_args
!python ./Time-Series-Library/run.py $args


In [ ]:
# Experiment with PatchTST
model = "PatchTST"
args = f"--model {model} --model_id {model}_BTC_lt " + cmn_args
!python ./Time-Series-Library/run.py $args

In [ ]:
# Experiment with DLinear
model = "DLinear"
args = f"--model {model} --model_id {model}_BTC_lt " + cmn_args
!python ./Time-Series-Library/run.py $args